## File names etc.

In [1]:
# angle_rep 0: Full unit circle, angle_rep 1: Right half plane, angle_rep 2: Regular angle
angle_rep = 0
img_list_file = '../list_of_img_in_val_set_18-03.csv'
#img_list_file = '../list_of_img_in_OP_val_set_14-04.csv'

anchor_config = './config.ini'

#path_img_folder = '../../../03 Data/Dataset2_onPallet/' Simple Dataset
path_img_folder = '../../../03 Data/Simple Dataset/' 


model_folder = "./02 On Pallet Dataset/01 Snapshots/"
model_name = "OP_Rotated_comb_norm_resnet50_csv_25.h5"

## Imports

In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)
# import keras_retinanet
#from keras_retinanet import models
from RetinaNet_Rotated_Comb.keras_retinanet import models

from RetinaNet_Rotated_Comb.keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from RetinaNet_Rotated_Comb.keras_retinanet.utils.visualization import draw_box, draw_caption
from RetinaNet_Rotated_Comb.keras_retinanet.utils.colors import label_color
from RetinaNet_Rotated_Comb.keras_retinanet.utils.gpu import setup_gpu
from RetinaNet_Rotated_Comb.keras_retinanet.utils.config import parse_anchor_parameters

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
#import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy

# use this to change which GPU to use
gpu = 0
# set the modified tf session as backend in keras  
#setup_gpu(gpu)  #NOTICE: enable when using paperspace server!!

Using TensorFlow backend.


['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/paperspace/HPM/HPMenv/lib/python3.6/site-packages', '/home/paperspace/HPM/HPMenv/lib/python3.6/site-packages/IPython/extensions', '/home/paperspace/.ipython', '/home/paperspace/HPM/BachelorProject/02 Deep Learning']


## Annotation Loading Functions

In [3]:
# Ouput a list of RotatedRects as annotations
def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        rot_rect_list= []
        for annotation in shapes_list: 
            #rect = cv2.boundingRect(np.float32(annotation['points']))
            rot_rect = cv2.minAreaRect(np.float32(annotation['points']))
            if rot_rect[-2][0] < rot_rect[-2][1]:
                rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
            # make sure that no angle is above 90 or below -90
            if rot_rect[-1] > 90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
            if rot_rect[-1] < -90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
            rot_rect_list.append(rot_rect)
    return rot_rect_list 

## Rectangle Functions

In [4]:
def rot_rect_area(rot_rect):
    width = rot_rect[1][0]
    height = rot_rect[1][1]
    return height * width

def rot_rect_intersection(rot_rect1, rot_rect2):
    check, points = cv2.rotatedRectangleIntersection(rot_rect1,rot_rect2)
    if check != 0:
        #hull = cv.convexHull(points)
        area = cv2.contourArea(points)
    else:
        area = 0
    return area

def rot_rect_union(rot_rect1, rot_rect2): 
    return rot_rect_area(rot_rect1) + rot_rect_area(rot_rect2) - rot_rect_intersection(rot_rect1, rot_rect2)

def rot_rect_IoU(rot_rect1, rot_rect2):
    return rot_rect_intersection(rot_rect1, rot_rect2)/rot_rect_union(rot_rect1, rot_rect2)

## Prediction Manipulation Functions

In [5]:
def filter_predictions(bboxes, scores, labels, angles, score_threshold):
    filtered_bboxes = []
    filtered_scores = []
    filtered_labels = []
    filtered_angles = []
    
    for bbox, score, label, angle in zip(bboxes, scores, labels, angles):
        if score < score_threshold: 
            break
            
        filtered_bboxes.append(bbox)
        filtered_scores.append(float(score))
        filtered_labels.append(label)
        filtered_angles.append(angle)

    return filtered_bboxes, filtered_scores, filtered_labels, filtered_angles

def format_angles(angles, rep):
    formatted_angles = []
    if rep == 0:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0])/2/np.pi*180)
    elif rep == 1:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0])/np.pi*180)
    elif rep == 2:
        formatted_angles.append(angle/np.pi*180)
    return formatted_angles 

def convert_to_rot_rect(bboxes, angles, rep):
    rot_rect_list = []
    f_angles = format_angles(angles, rep)
    for box, angle in zip (bboxes, f_angles):
        center_x = (box[0]+box[2])/2
        center_y = (box[1]+box[3])/2
        w = abs(box[2]-box[0])
        h = abs(box[3]-box[1])
        rot_rect_list.append(((center_x, center_y),(w, h), angle))
    return rot_rect_list

## Evaluation Functions

In [6]:
# creates a matrix of IoU between prediction and annotation rectangles. The predictions and annotations with the highest IoU
# are matched. The matching indices and associated IoU-scores are reported as (pred_idx, anno_idx, IoU)
def match_annotations(rot_rect_predictions, rot_rect_annotations):
    #create the matrix
    IoU_mat = np.zeros((len(rot_rect_predictions), len(rot_rect_annotations)))
    # fill matrix with IoU values
    for pred_idx, pred in enumerate(rot_rect_predictions):
        for anno_idx, anno in enumerate(rot_rect_annotations):
            IoU_mat[pred_idx, anno_idx] = rot_rect_IoU(pred, anno)
    
    # create output list
    annotation_matches = []
    if IoU_mat.size != 0:
        max_idx = np.unravel_index(np.argmax(IoU_mat, axis=None), IoU_mat.shape)
        max_IoU = IoU_mat[max_idx]
        while max_IoU > 0:
            # set the chosen rows and columns to 0
            IoU_mat[max_idx[0],:] = 0
            IoU_mat[:,max_idx[1]] = 0
            # append the indices and IoU to result
            annotation_matches.append((max_idx[0],max_idx[1],max_IoU))
            max_idx = np.unravel_index(np.argmax(IoU_mat, axis=None), IoU_mat.shape)
            max_IoU = IoU_mat[max_idx]
    
    return annotation_matches

# Returns (T. Pos, F. Pos, F. Neg) for certain threshold
def evaluate_bb(predictions, annotations, annotation_matches, threshold):
    i = 0
    true_pos = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        i+=1
        true_pos+=1
    false_pos = len(predictions) - true_pos;
    false_neg = len(annotations) - true_pos;
    return true_pos, false_pos, false_neg

# Returns the sum of angle errors and squared sum of angle errors for true positives
# Errors are assumed in degrees
def evaluate_angle(predictions, annotations, annotation_matches, threshold):
    i = 0
    angle_err = 0
    angle_err_sqr = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        pred_idx, anno_idx = annotation_matches[i][0:-1]
        diff = annotations[anno_idx][-1] - predictions[pred_idx][-1] # using the last element of the tuple, which is the angle
        diff = min(abs(diff), abs(diff-180), abs(diff+180)) # makes sure that the sigularity does not skew the results. Assumes degrees
        angle_err += diff
        angle_err_sqr += diff**2
        i+=1
    return angle_err, angle_err_sqr

def evaluate_center_point(predictions, annotations, annotation_matches, threshold):
    i = 0
    center_pt_err = 0
    center_pt_err_sqr = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        pred_idx, anno_idx = annotation_matches[i][0:-1]
        diff = np.sqrt((annotations[anno_idx][0][0] - predictions[pred_idx][0][0])**2 + (annotations[anno_idx][0][1] - predictions[pred_idx][0][1])**2) 
        center_pt_err += diff
        center_pt_err_sqr += diff**2
        i+=1
    return center_pt_err, center_pt_err_sqr

# finds annotation matches and calls evaluate_bb and evaluate_angle for several IoU-thresholds
# ongoing_results is a tuple (total_T_pos, total_F_pos, total_F_neg, angle_err_sum, angle_err_sqr_sum, center_pt_err_sum, center_pt_err_sqr_sum)
def evaluate_range(ongoing_results, rot_rect_preds, rot_rect_annos, thresholds):
    annotation_matches = match_annotations(rot_rect_preds, rot_rect_annos)
    for idx, thresh in enumerate(thresholds):
        true_pos, false_pos, false_neg = evaluate_bb(rot_rect_preds, rot_rect_annos, annotation_matches, thresh)
        angle_err, angle_err_sqr = evaluate_angle(rot_rect_preds, rot_rect_annos, annotation_matches, thresh)
        center_pt_err, center_pt_err_sqr = evaluate_center_point(rot_rect_preds, rot_rect_annos, annotation_matches, thresh)
        ongoing_results[idx][0] += true_pos 
        ongoing_results[idx][1] += false_pos
        ongoing_results[idx][2] += false_neg
        ongoing_results[idx][3] += angle_err
        ongoing_results[idx][4] += angle_err_sqr
        ongoing_results[idx][5] += center_pt_err
        ongoing_results[idx][6] += center_pt_err_sqr
        
    return ongoing_results

#Returns tuple (thresh, prec, rec, f1, avg_ang_err, std_ang_err, avg_center_err, std_center_err, t_pos, f_pos, f_neg)
def get_result(final_results, thresholds):
    result = []
    avg_prec = 0
    avg_rec = 0
    avg_avg_ang_err = 0
    avg_std_ang_err = 0
    avg_avg_center_err = 0
    avg_std_center_err = 0
    for final_res, thresh in zip(final_results, thresholds):
        precision = final_res[0]/(final_res[0]+final_res[1])
        recall = final_res[0]/(final_res[0]+final_res[2])
        f1 = 2*precision*recall/(precision+recall)
        avg_ang_err = final_res[3]/final_res[0]
        std_ang_err = np.sqrt((final_res[4]/final_res[0])-(avg_ang_err**2))
        avg_center_err = final_res[5]/final_res[0]
        std_center_err = np.sqrt((final_res[6]/final_res[0])-(avg_center_err**2))
        result.append((thresh,precision, recall, f1, avg_ang_err, std_ang_err, avg_center_err, std_center_err, final_res[0], final_res[1], final_res[2]))
        avg_prec += precision
        avg_rec += recall
        avg_avg_ang_err += avg_ang_err
        avg_std_ang_err += std_ang_err
        avg_avg_center_err += avg_center_err
        avg_std_center_err += std_center_err
    # append the average at the end 
    l = len(thresholds)
    result.append(('Avg', avg_prec/l, avg_rec/l, 2*avg_prec/l*avg_rec/l/(avg_prec/l+avg_rec/l), avg_avg_ang_err/l, avg_std_ang_err/l,avg_avg_center_err/l, avg_std_center_err/l, '--','--','--'))
    return result
    

## File Generation

In [7]:
def write_results(folder, name, results, time):
    with open(folder + name, 'w') as f:
        f.write('IoU, Prec., Rec., F1, ang. err., std. ang. err., cent. err., std. cent. err. ,F. Neg, F. Pos, T. Pos \n')
        for result in results:
            f.write(str(result[0])+', '+str(result[1])+', '+str(result[2])+', '+str(result[3])+', '+str(result[4])+', '+str(result[5])+ ', '+str(result[6])+', '+str(result[7])+', '+str(result[10])+', '+str(result[9])+', '+str(result[8])+'\n')
        f.write('Time: '+str(time))

## Visualisation

In [8]:
def draw_predictions(rot_rect_preds, img, save_loc):
    for rect in rot_rect_preds:
        box = cv2.boxPoints(rect) 
        box = np.int0(box)
        cv2.drawContours(img,[box],0,(0,0,255),2)
    cv2.imwrite(save_loc, img)
    return img

## Model Setup

In [9]:
# import model
model_path = os.path.join(model_folder, model_name);
model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
#anchor_params = parse_anchor_parameters(anchor_config)
model = models.convert_model(model, nms=False);

# Mapping of model output and classes
labels_to_names = {0: 'Brick'};

tracking <tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
array([[-32.    ,  -8.    ,  32.    ,   8.    ],
       [-40.3168, -10.0792,  40.3168,  10.0792],
       [-50.7968, -12.6992,  50.7968,  12.6992]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
array([[ -64.    ,  -16.    ,   64.    ,   16.    ],
       [ -80.6336,  -20.1584,   80.6336,   20.1584],
       [-101.5936,  -25.3984,  101.5936,   25.3984]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
array([[-128.    ,  -32.    ,  128.    ,   32.    ],
       [-161.2672,  -40.3168,  161.2672,   40.3168],
       [-203.1872,  -50.7968,  203.1872,   50.7968]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
array([[-256.    ,  -64.    ,  256.    ,   64.    ],
       [-322.5344,  -80.6336,  322.5344,   80.6336],
       [-406.3744, -101.5936,  406.3744,  101.5936]], dtype=float32)> anch

## Evaluation

In [10]:
# import data
file = open(img_list_file)
file_paths = list(file)
#file_paths = file_paths[0:30]

IoU_thresholds = np.arange(0.5,1,0.05)

ongoing_results = np.zeros((len(IoU_thresholds),7))

total_time = 0
for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    start = time.perf_counter()
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    # process image 
    boxes, scores, labels, angles = model.predict_on_batch(np.expand_dims(image, axis=0))
    # correct for image scale  
    boxes /= scale
    boxes, scores, labels, angles = filter_predictions(boxes[0], scores[0], labels[0], angles[0], 0.9)
    
    rot_rect_preds = convert_to_rot_rect(boxes, angles, angle_rep)
    #do some nms
    
    indices=cv2.dnn.NMSBoxesRotated(rot_rect_preds, scores, 0.2, 0.15)
    indices = list(map(int,indices))
        
    print(len(indices))
    rot_rect_preds = [rot_rect_preds[x] for x in indices]
    scores = [scores[x] for x in indices]
    labels = [labels[x] for x in indices]
    end = time.perf_counter()
    total_time += end-start
    
    ##load annotations
    rot_rect_annos = read_annotations_from_json(path.strip('\n')) 
    
    img = draw_predictions(rot_rect_preds, draw, './02 On Pallet Dataset/02 Evaluation Images/'+path.strip('\n'))
    
    ongoing_results =  evaluate_range(ongoing_results, rot_rect_preds, rot_rect_annos, IoU_thresholds) # <- Edit to use RotatedRect
final_results = get_result(ongoing_results, IoU_thresholds)
write_results('./02 On Pallet Dataset/', model_name[0:-3]+'results_simple.csv', final_results, total_time)    

11
8
16
11
15
15
13
13
15
15
15
14
13
14
14
7
16
16
14
15
16
16
15
14
16
12
16
16
15
15
14
16
16
16
15
16
16
16
15
16


In [11]:
print(angles)

[array([-0.8935869 , -0.43153107], dtype=float32), array([-1.0186518 , -0.37874824], dtype=float32), array([-0.89156634, -0.39893797], dtype=float32), array([-0.8804049, -0.401723 ], dtype=float32), array([-0.8783738 , -0.44939083], dtype=float32), array([-0.943411  , -0.35680312], dtype=float32), array([-0.9660763 , -0.33796355], dtype=float32), array([-0.8934555 , -0.42526776], dtype=float32), array([-0.9551879 , -0.40151384], dtype=float32), array([-1.0217218 , -0.39939684], dtype=float32), array([-0.8969149, -0.5202827], dtype=float32), array([-0.9333686 , -0.37796167], dtype=float32), array([-0.9070306, -0.4518152], dtype=float32), array([-0.9036355 , -0.40104666], dtype=float32), array([-0.8420044 , -0.36035138], dtype=float32), array([-0.89452016, -0.45649177], dtype=float32), array([-0.89242154, -0.47438556], dtype=float32), array([-0.88440984, -0.4029983 ], dtype=float32), array([-0.8905921 , -0.43858784], dtype=float32), array([-0.92299205, -0.5381268 ], dtype=float32), array